In [ ]:
!pip install python-terrier
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import nltk
import pyterrier as pt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.stem import *
from nltk.stem.porter import *
from collections import Counter
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
pd.set_option('display.max_colwidth', 150)

**Test Collection**

In [ ]:
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [ ]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")
topics = vaswani_dataset.get_topics()
topics.head(5)
qrels = vaswani_dataset.get_qrels()


query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

In [ ]:
# Vaswani Dataset
vaswani_dataset = pt.datasets.get_dataset("vaswani")
topics = vaswani_dataset.get_topics()
qrels = vaswani_dataset.get_qrels()
data_path = vaswani_dataset.get_corpus()[0]

doc-text.trec:   0%|          | 0.00/0.99M [00:00<?, ?iB/s]

In [ ]:
with open(data_path, 'r') as file:
    documents = file.read()

documents = documents.split('<DOCNO>')
df = pd.DataFrame(documents)
# the documents stored in the vaswani_dataset
df['documents'] = df
df.drop(0, axis = 1, inplace = True)

In [ ]:
df

,documents
0,<DOC>\n
1,1</DOCNO>\ncompact memories have flexible capacities a digital data storage\nsystem with capacity up to bits and random and or sequential access\...
2,2</DOCNO>\nan electronic analogue computer for solving systems of linear equations\nmathematical derivation of the operating principle and stabili...
3,3</DOCNO>\nelectronic coordinate transformer circuit details are given for\nthe construction of an electronic calculating unit which enables\nthe...
4,4</DOCNO>\nthe british computer society report of a conference held in cambridge\njune\n</DOC>\n<DOC>\n
...,...
11425,11425</DOCNO>\ndiurnal power variation of the earth ionosphere cavity modes and\ntheir relationship to world wide thunderstorm activity\n</DOC>\n<...
11426,11426</DOCNO>\non the design of small economical radio frequency eht supplies particular\nattention is given to the design of the high voltage tr...
11427,11427</DOCNO>\nsatellite power supply has variable pulse width a detailed circuit\ndescription of a short circuit proof dc to dc converter of hig...
11428,11428</DOCNO>\nsolar batteries for the use as the power source of unattended uhf\nrepeaters\n</DOC>\n<DOC>\n


In [ ]:
topics

,qid,query
0,1,measurement of dielectric constant of liquids by the use of microwave techniques
1,2,mathematical analysis and design details of waveguide fed microwave radiations
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of computers
...,...,...
88,89,tunnel diode construction and its electrical characteristics explained
89,90,electronic density of states at the surface of a semiconductor compared with that at depth
90,91,resistivity of metallic thin films related to surface roughness
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields


In [ ]:
qrels

,qid,docno,label
0,1,1239,1
1,1,1502,1
2,1,4462,1
3,1,4569,1
4,1,5472,1
...,...,...,...
2078,93,9875,1
2079,93,9956,1
2080,93,10497,1
2081,93,11191,1


**Preprocessing**

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def preprocessing(text):
  #cleaning
    text = str(text)  # Convert text to a string
    text = re.sub(r"http\S+", " ", text)  # Remove URLs
    text = re.sub(r"RT ", " ", text)  # Remove RT
    text = re.sub(r"@[\w]*", " ", text)  # Remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text)  # Remove special characters
    text = re.sub(r'\t', ' ', text)  # Remove tabs
    text = re.sub(r'\n', ' ', text)  # Remove line jumps
    text = re.sub(r"\s+", " ", text)  # Remove extra white spaces
    text = text.strip()
    text = re.sub(r" < doc > < doc >", " ", text)
    text = re.sub(r"< docno > ", " ", text)
    tokens=word_tokenize(text)
    remove_stopwords=[word.lower()for word in tokens if word.lower()not in stop_words]
    stemmed=[stemmer.stem(word) for word in remove_stopwords]
    return " ".join(stemmed)




In [ ]:
df['processed_documents'] = df['documents'].apply(preprocessing)
df['processed_documents']  = df['processed_documents'].replace(to_replace=r'\d', value='', regex=True)  # this line removes any digit found in the text
df['documents']  = df['documents'].replace(to_replace=r'\d', value='', regex=True)

**Indexing**

In [ ]:
# pd_indexer = pt.DFIndexer("./pd_index",overwrite=True)
# index = pt.IndexFactory.of(indexref)
# print(index.getCollectionStatistics().toString())
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [ ]:
def doc(df):
  df['docno']=range(len(df))
  df['docno']=df['docno'].astype(str)
  return df
df = doc(df)
df

,documents,processed_documents,docno
0,<DOC>\n,< doc >,0
1,</DOCNO>\ncompact memories have flexible capacities a digital data storage\nsystem with capacity up to bits and random and or sequential access\n...,< docno > compact memori flexibl capac digit data storag system capac bit random sequenti access describ < doc > < doc >,1
2,</DOCNO>\nan electronic analogue computer for solving systems of linear equations\nmathematical derivation of the operating principle and stabilit...,< docno > electron analogu comput solv system linear equat mathemat deriv oper principl stabil condit comput consist amplifi < doc > < doc >,2
3,</DOCNO>\nelectronic coordinate transformer circuit details are given for\nthe construction of an electronic calculating unit which enables\nthe ...,< docno > electron coordin transform circuit detail given construct electron calcul unit enabl polar coordin vector modulu cosin sine argument de...,3
4,</DOCNO>\nthe british computer society report of a conference held in cambridge\njune\n</DOC>\n<DOC>\n,< docno > british comput societi report confer held cambridg june < doc > < doc >,4
...,...,...,...
11425,</DOCNO>\ndiurnal power variation of the earth ionosphere cavity modes and\ntheir relationship to world wide thunderstorm activity\n</DOC>\n<DOC>\n,< docno > diurnal power variat earth ionospher caviti mode relationship world wide thunderstorm activ < doc > < doc >,11425
11426,</DOCNO>\non the design of small economical radio frequency eht supplies particular\nattention is given to the design of the high voltage transfo...,< docno > design small econom radio frequenc eht suppli particular attent given design high voltag transform type stabil eht gener describ suitab...,11426
11427,</DOCNO>\nsatellite power supply has variable pulse width a detailed circuit\ndescription of a short circuit proof dc to dc converter of high eff...,< docno > satellit power suppli variabl puls width detail circuit descript short circuit proof dc dc convert high effici input voltag output volt...,11427
11428,</DOCNO>\nsolar batteries for the use as the power source of unattended uhf\nrepeaters\n</DOC>\n<DOC>\n,< docno > solar batteri use power sourc unattend uhf repeat < doc > < doc >,11428


In [ ]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
# index the text, record the docnos as metadata
index_ref = indexer.index(df["processed_documents"], df["docno"])
index_ref.toString()

11:30:13.960 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (0) - further warnings are suppressed
11:30:36.116 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


'./DatasetIndex/data.properties'

In [ ]:
index = pt.IndexFactory.of(index_ref)

In [ ]:
doc_index = index.getDocumentIndex()


In [ ]:
print(index.getCollectionStatistics())


Number of documents: 11430
Number of terms: 7706
Number of postings: 226290
Number of fields: 0
Number of tokens: 273326
Field names: []
Positions:   false



In [ ]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString().split('@')[0]))


Streaming output truncated to the last 5000 lines.
gossard -> term2734 Nt=1 TF=1 maxTF=1  
gottingen -> term6289 Nt=1 TF=1 maxTF=1  
gould -> term7227 Nt=1 TF=1 maxTF=1  
govenr -> term6593 Nt=1 TF=1 maxTF=1  
govern -> term351 Nt=28 TF=28 maxTF=1  
gpa -> term5971 Nt=1 TF=1 maxTF=1  
graaff -> term6667 Nt=2 TF=2 maxTF=1  
grad -> term6628 Nt=1 TF=1 maxTF=1  
grade -> term374 Nt=9 TF=9 maxTF=1  
gradient -> term1190 Nt=67 TF=82 maxTF=3  
gradual -> term2250 Nt=12 TF=12 maxTF=1  
graduat -> term4614 Nt=3 TF=3 maxTF=1  
graffund -> term6760 Nt=1 TF=1 maxTF=1  
grahamstown -> term1657 Nt=2 TF=2 maxTF=1  
grai -> term1702 Nt=2 TF=2 maxTF=1  
grain -> term3355 Nt=6 TF=7 maxTF=2  
gram -> term5188 Nt=2 TF=3 maxTF=2  
gramophon -> term4959 Nt=4 TF=6 maxTF=3  
grand -> term7630 Nt=1 TF=1 maxTF=1  
granit -> term3926 Nt=1 TF=1 maxTF=1  
granular -> term4677 Nt=2 TF=2 maxTF=1  
graph -> term731 Nt=77 TF=89 maxTF=3  
graphic -> term1494 Nt=191 TF=200 maxTF=2  
graphit -> term4464 Nt=3 TF=3 maxTF=

In [ ]:
words_list = []
for kv in index.getLexicon():
  words_list.append(kv.getKey())
print(words_list)


['aa', 'aachen', 'aad', 'aan', 'ab', 'abac', 'abandon', 'abbrevi', 'abd', 'abel', 'aber', 'aberr', 'abil', 'abl', 'ablat', 'abnorm', 'aboard', 'abomb', 'abrad', 'abraham', 'abridg', 'abrupt', 'abruptli', 'abscissa', 'absenc', 'absent', 'absolut', 'absorb', 'absorpt', 'abstract', 'abu', 'abund', 'ac', 'academi', 'accel', 'acceleromet', 'accentu', 'accept', 'acceptor', 'access', 'accessori', 'accid', 'accommod', 'accomod', 'accompani', 'accomplish', 'accord', 'accordingli', 'account', 'accret', 'accross', 'accumul', 'accur', 'accuraci', 'acetaldehyd', 'aceton', 'achiev', 'achimota', 'acid', 'acknowledg', 'acm', 'acoust', 'acousto', 'acoustoelectr', 'acquir', 'across', 'act', 'action', 'activ', 'actual', 'actuat', 'acut', 'ad', 'ada', 'adam', 'adapt', 'adcock', 'add', 'addam', 'addendum', 'adder', 'addit', 'address', 'adduc', 'adelaid', 'adequ', 'adh', 'adiabat', 'adjac', 'adjoin', 'adjoint', 'adjust', 'adjustabl', 'adler', 'administr', 'admir', 'admira', 'admitt', 'admixtur', 'adopt', 'a

**Query Processing**

In [ ]:
Query = "resistivity of metallic thin films related to surface roughness"
processed_Query = preprocessing(Query)
Query_terms = word_tokenize(processed_Query)

postings_list = []

for term in Query_terms:
    try:
        word = index.getLexicon()[term]
        postings = index.getInvertedIndex().getPostings(word)
        postings_list.append((term, postings))
    except KeyError:
        print("Term not found" % term)

# Print postings for each term
for term, postings in postings_list:
    print("Postings for term '%s':" % term)
    for p in postings:
        print(p.toString())


Postings for term 'resist':
ID(34) TF(1)
ID(45) TF(1)
ID(46) TF(1)
ID(54) TF(1)
ID(171) TF(1)
ID(173) TF(1)
ID(274) TF(1)
ID(279) TF(1)
ID(280) TF(1)
ID(294) TF(1)
ID(301) TF(1)
ID(411) TF(1)
ID(416) TF(1)
ID(417) TF(2)
ID(420) TF(1)
ID(429) TF(2)
ID(493) TF(1)
ID(522) TF(1)
ID(529) TF(1)
ID(541) TF(1)
ID(570) TF(1)
ID(631) TF(1)
ID(634) TF(1)
ID(650) TF(1)
ID(681) TF(1)
ID(771) TF(1)
ID(781) TF(1)
ID(785) TF(2)
ID(917) TF(2)
ID(919) TF(1)
ID(920) TF(1)
ID(925) TF(1)
ID(928) TF(1)
ID(929) TF(2)
ID(944) TF(1)
ID(951) TF(1)
ID(955) TF(1)
ID(970) TF(1)
ID(1060) TF(1)
ID(1066) TF(1)
ID(1075) TF(1)
ID(1085) TF(2)
ID(1230) TF(1)
ID(1323) TF(1)
ID(1331) TF(2)
ID(1460) TF(1)
ID(1462) TF(1)
ID(1463) TF(1)
ID(1471) TF(1)
ID(1472) TF(2)
ID(1476) TF(1)
ID(1478) TF(1)
ID(1488) TF(1)
ID(1515) TF(1)
ID(1596) TF(2)
ID(1600) TF(3)
ID(1608) TF(1)
ID(1615) TF(1)
ID(1626) TF(3)
ID(1640) TF(1)
ID(1719) TF(1)
ID(1751) TF(1)
ID(1824) TF(1)
ID(1840) TF(1)
ID(1843) TF(1)
ID(1848) TF(1)
ID(1863) TF(1)
ID(1876) 

In [ ]:
df

,documents,processed_documents,docno
0,<DOC>\n,< doc >,0
1,</DOCNO>\ncompact memories have flexible capacities a digital data storage\nsystem with capacity up to bits and random and or sequential access\n...,< docno > compact memori flexibl capac digit data storag system capac bit random sequenti access describ < doc > < doc >,1
2,</DOCNO>\nan electronic analogue computer for solving systems of linear equations\nmathematical derivation of the operating principle and stabilit...,< docno > electron analogu comput solv system linear equat mathemat deriv oper principl stabil condit comput consist amplifi < doc > < doc >,2
3,</DOCNO>\nelectronic coordinate transformer circuit details are given for\nthe construction of an electronic calculating unit which enables\nthe ...,< docno > electron coordin transform circuit detail given construct electron calcul unit enabl polar coordin vector modulu cosin sine argument de...,3
4,</DOCNO>\nthe british computer society report of a conference held in cambridge\njune\n</DOC>\n<DOC>\n,< docno > british comput societi report confer held cambridg june < doc > < doc >,4
...,...,...,...
11425,</DOCNO>\ndiurnal power variation of the earth ionosphere cavity modes and\ntheir relationship to world wide thunderstorm activity\n</DOC>\n<DOC>\n,< docno > diurnal power variat earth ionospher caviti mode relationship world wide thunderstorm activ < doc > < doc >,11425
11426,</DOCNO>\non the design of small economical radio frequency eht supplies particular\nattention is given to the design of the high voltage transfo...,< docno > design small econom radio frequenc eht suppli particular attent given design high voltag transform type stabil eht gener describ suitab...,11426
11427,</DOCNO>\nsatellite power supply has variable pulse width a detailed circuit\ndescription of a short circuit proof dc to dc converter of high eff...,< docno > satellit power suppli variabl puls width detail circuit descript short circuit proof dc dc convert high effici input voltag output volt...,11427
11428,</DOCNO>\nsolar batteries for the use as the power source of unattended uhf\nrepeaters\n</DOC>\n<DOC>\n,< docno > solar batteri use power sourc unattend uhf repeat < doc > < doc >,11428


In [ ]:
Query = "resistivity of metallic thin films related to surface roughness"
processed_Query = preprocessing(Query)
tokenized = word_tokenize(processed_Query)
Relvent_documents = []
for word in tokenized:
 if word in words_list:
  for lexicon in index.getInvertedIndex().getPostings(index.getLexicon()[word]):
      Relvent_documents.append(lexicon.getId())



In [ ]:
print("\033[3;4;1mThe Relevant Documents to this query\033[0m\n")
for i in range(len(Relvent_documents)):
    print(df["processed_documents"][Relvent_documents[i]])
    print("\n")


Streaming output truncated to the last 5000 lines.


 < docno > diffract puls resist half plane two dimension diffract problem acoust em incid puls front parallel edg half plane solv assum dynam similar solu ion relat problem three dimension problem aris plane puls incid obliqu edg resist half plane discuss < doc > < doc >


 < docno > valid f concept specif surfac resist measur extens earlier work lacost show tht criterion surfac conduct independ electrod system configur influenc configur accuraci measur consid < doc > < doc >


 < docno > neg resist concept modern electron survey particular refer oper semiconductor diod microwav valv < doc > < doc >


 < docno > iter network coupl network oscil circuit condit self oscil establish valv transistor oscil characterist iter network consist pure resist reactiv element summar applic valv transistor oscil investig formula given oper paramet verifi experiment < doc > < doc >


 < docno > theori impur resist metal mani bodi techniqu develop ca

In [ ]:
Query = "resistivity of metallic thin films related to surface roughness"      # 3ayza ab2a atb3 el query
processed_Query = preprocessing(Query)
tokenized = word_tokenize(processed_Query)
tibical_documents = []

# Initialize a list for each term in the query
term_lists = {word: [] for word in tokenized}

# Populate the lists with document IDs containing each term
for word in tokenized:
    if word in words_list:
        for lexicon in index.getInvertedIndex().getPostings(index.getLexicon()[word]):
            term_lists[word].append(lexicon.getId())

# Initialize all_list
all_list = []

# Loop through the values of the dictionary and extend all_list
for sublist in term_lists.values():
    all_list.append(sublist)

print("All list:", all_list)



All list: [[34, 45, 46, 54, 171, 173, 274, 279, 280, 294, 301, 411, 416, 417, 420, 429, 493, 522, 529, 541, 570, 631, 634, 650, 681, 771, 781, 785, 917, 919, 920, 925, 928, 929, 944, 951, 955, 970, 1060, 1066, 1075, 1085, 1230, 1323, 1331, 1460, 1462, 1463, 1471, 1472, 1476, 1478, 1488, 1515, 1596, 1600, 1608, 1615, 1626, 1640, 1719, 1751, 1824, 1840, 1843, 1848, 1863, 1876, 1977, 2020, 2064, 2067, 2071, 2091, 2185, 2295, 2308, 2319, 2332, 2335, 2366, 2396, 2426, 2427, 2544, 2670, 2719, 2792, 2808, 2816, 2835, 2885, 2886, 2942, 2945, 2946, 2972, 3038, 3059, 3136, 3158, 3261, 3265, 3390, 3392, 3473, 3508, 3544, 3608, 3628, 3689, 3715, 3719, 3749, 3753, 3791, 3845, 3859, 3876, 3877, 3886, 3948, 3961, 3963, 3977, 4053, 4054, 4064, 4096, 4181, 4319, 4324, 4331, 4440, 4525, 4538, 4549, 4555, 4590, 4591, 4602, 4607, 4613, 4619, 4639, 4709, 4713, 4714, 4731, 4751, 4815, 4834, 4839, 4844, 4856, 4928, 4929, 4930, 4932, 4933, 4936, 4944, 4969, 5023, 5043, 5044, 5053, 5066, 5127, 5155, 5194, 5202

In [ ]:
from collections import Counter

# Merge lists
merged_list = sum( all_list, [])

# Count occurrences
counted = Counter(merged_list)

# Identify common numbers
common_numbers = [num for num, count in counted.items() if count == len( all_list)]

print("Common numbers:", common_numbers)


Common numbers: []


**TF-IDF**

In [ ]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)


In [ ]:
query = preprocessing("resistivity of metallic thin films related to surface roughness")
results=tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,6581,6581,0,16.484710,resist metal thin film relat surfac rough
1,1,7594,7594,1,16.297074,resist metal thin film relat surfac rough
2,1,9991,9991,2,14.695017,resist metal thin film relat surfac rough
3,1,4751,4751,3,14.671036,resist metal thin film relat surfac rough
4,1,7603,7603,4,14.074281,resist metal thin film relat surfac rough
5,1,9224,9224,5,13.846516,resist metal thin film relat surfac rough
6,1,6182,6182,6,13.460613,resist metal thin film relat surfac rough
7,1,2854,2854,7,13.407825,resist metal thin film relat surfac rough
8,1,1992,1992,8,12.963975,resist metal thin film relat surfac rough
9,1,9864,9864,9,12.739125,resist metal thin film relat surfac rough


In [ ]:
#get the 3 top retrived documents
top_answer = df[df["docno"].isin(['6581', '7594','9991'])]['processed_documents']
top_answer

6581     < docno > physic thin metal semiconductor film survey work format structur electr properti thin metal semiconductor film surfac state effect prop...
7594                                             < docno > resist imperfect contact two metal comparison experiment result thin granular film < doc > < doc >
9991     < docno > determin thick optic constant thin film theori given relat thick optic constant thin metal film deposit glass reflect transmit light in...
Name: processed_documents, dtype: object

**Query Expansion**

In [ ]:
query = preprocessing("resistivity")
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results


,qid,docid,docno,rank,score,query
0,1,6724,6724,0,8.094763,resist
1,1,9664,9664,1,7.815008,resist
2,1,1596,1596,2,7.331773,resist
3,1,1066,1066,3,7.236953,resist
4,1,9162,9162,4,7.233230,resist
5,1,2808,2808,5,7.139113,resist
6,1,2792,2792,6,7.049181,resist
7,1,8853,8853,7,7.049181,resist
8,1,2835,2835,8,7.046531,resist
9,1,4714,4714,9,7.046531,resist


In [ ]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]
expanded_query

'applypipeline:off neg^0.068379752 resist^0.742593348 design^0.024513498 comput^0.024513498 constant^0.025803683 network^0.030641876 dc^0.024513498 potentiomet^0.022940468 function^0.016747583 amplifi^0.019352762'

In [ ]:
print(expanded_query)

applypipeline:off neg^0.068379752 resist^0.742593348 design^0.024513498 comput^0.024513498 constant^0.025803683 network^0.030641876 dc^0.024513498 potentiomet^0.022940468 function^0.016747583 amplifi^0.019352762


In [ ]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)


neg^0.068379752
resist^0.742593348
design^0.024513498
comput^0.024513498
constant^0.025803683
network^0.030641876
dc^0.024513498
potentiomet^0.022940468
function^0.016747583
amplifi^0.019352762

resist


In [ ]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))

#Let's check the tweets text for the top 5 retrieved tweets
df[['processed_documents']][df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1   score_1  docid_2   score_2
0     6724  8.094763     2808  8.546838
1     9664  7.815008     2792  8.423169
2     1596  7.331773     8853  8.190733
3     1066  7.236953     6724  8.094763
4     9162  7.233230     2835  8.004549


,processed_documents
1596,< docno > resist potentiomet function gener gener function singl variabl mean resist potentiomet review see also < doc > < doc >
2792,< docno > neg resist dc comput < doc > < doc >
2808,< docno > neg resist dc comput nomogram present design circuit use transistor resistor give requir valu neg resist < doc > < doc >
2835,< docno > role neg effect mass neg resist theoret investig relat neg effect mass carrier distribut neg resist < doc > < doc >
6724,< docno > mechan resist variat magnet field model two layer differ resist resist propos explain hall effect variat resist materi subject electr f...
8853,< docno > neg resist amplifi design < doc > < doc >


**Elmo**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [ ]:
sentences = ["economical relay operated accumulator design details are given of a binary adder with a maximum capacity of about","economical aspects of printed foil circuits"]
embeddings = elmo.signatures["default"](tf.constant(sentences))["elmo"]
first_sentence_embedding = embeddings.numpy()[0][0]
print(first_sentence_embedding)
sec_sentence_embedding = embeddings.numpy()[1][0]
print(sec_sentence_embedding)

#calculate cosine similarity between the embeddings
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

#compare embeddings
similarity_score = cosine_similarity(first_sentence_embedding, sec_sentence_embedding)
print(similarity_score)

[-0.09534319  0.16789994  0.32778534 ... -0.01206462  0.4103365
  0.68753105]
[-0.09534319  0.16789994  0.32778534 ... -0.12227698  0.11535341
  0.5126012 ]
0.8128229


**bert**

In [ ]:
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git--quiet


  Cloning https://github.com/experimaestro/experimaestro-ir.git--quiet to /tmp/pip-req-build-gi6f0_2s
  Running command git clone --filter=blob:none --quiet https://github.com/experimaestro/experimaestro-ir.git--quiet /tmp/pip-req-build-gi6f0_2s
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/experimaestro/experimaestro-ir.git--quiet /tmp/pip-req-build-gi6f0_2s did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/experimaestro/experimaestro-ir.git--quiet /tmp/pip-req-build-gi6f0_2s did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

model_name = "bert-base-uncased"
# Load the BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
for doc_id in results_wqe['docno'].loc[0:5]:
    document_text = df.loc[df['docno'] == doc_id, 'documents'].iloc[0] # Retrieve text of current document
    tokenized_text = bert_tokenizer.tokenize(document_text) # Tokenize document text
    token_ids = bert_tokenizer.convert_tokens_to_ids(tokenized_text) # Convert tokens to token IDs
    tokens_tensor = torch.tensor([token_ids]) # Convert token IDs to tensor

    with torch.no_grad():
        outputs = bert_model(tokens_tensor)
    query_embeddings = outputs[0][0].numpy()  # Use the output of the first layer
    print('BERT embeddings for document', doc_id, ': ', query_embeddings)

BERT embeddings for document 2808 :  [[ 0.23338361 -1.1012442  -0.02498751 ... -0.5531843  -1.1214204
   0.64818716]
 [ 0.22865793 -1.1028447   0.02743358 ... -0.45111412 -1.5303894
   0.5103711 ]
 [ 0.39291608 -1.196555    0.48655072 ... -0.53679144 -0.82226026
   0.58774465]
 ...
 [ 0.40552443 -1.1334218   0.2074331  ... -0.20208228 -1.5974869
   0.3508245 ]
 [ 0.3265556  -1.2256693   0.6187653  ... -0.4479923  -1.1428645
   0.57191277]
 [ 1.0308053  -1.1302725   0.35243663 ... -0.5726559  -1.3748499
   0.4729847 ]]
BERT embeddings for document 2792 :  [[ 0.27584913 -0.40336943 -0.72823465 ... -0.75757456 -0.42380223
   0.7998747 ]
 [ 0.4850481  -0.5721358  -0.60109884 ... -0.67596096 -1.307654
   0.4469099 ]
 [ 0.6425882  -0.28520915 -0.24950847 ... -0.59636956 -0.45740342
   0.83666575]
 ...
 [ 0.6329887  -0.74501556 -0.42213774 ... -0.45889705 -1.4407598
   0.43121907]
 [ 0.7912994  -0.7511356  -0.22475748 ... -0.52899146 -0.88803613
   0.64136535]
 [ 1.0603123  -0.6812305  -0.227

**Evaluation**

In [ ]:
#specify BM25 as wmodel
bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=10)   #work by bm25 l2no by4t8l b el dynamic ranking

bm25_res=bm25_retr.transform(topics)
bm25_res.head(11)

,qid,docid,docno,rank,score,query
0,1,8172,8172,0,24.614757,measurement of dielectric constant of liquids by the use of microwave techniques
1,1,9881,9881,1,22.140010,measurement of dielectric constant of liquids by the use of microwave techniques
2,1,5502,5502,2,21.764363,measurement of dielectric constant of liquids by the use of microwave techniques
3,1,1502,1502,3,19.497202,measurement of dielectric constant of liquids by the use of microwave techniques
4,1,4871,4871,4,18.341602,measurement of dielectric constant of liquids by the use of microwave techniques
5,1,4817,4817,5,18.155349,measurement of dielectric constant of liquids by the use of microwave techniques
6,1,9859,9859,6,17.926671,measurement of dielectric constant of liquids by the use of microwave techniques
7,1,8276,8276,7,17.479300,measurement of dielectric constant of liquids by the use of microwave techniques
8,1,7234,7234,8,16.616113,measurement of dielectric constant of liquids by the use of microwave techniques
9,1,720,720,9,16.402371,measurement of dielectric constant of liquids by the use of microwave techniques


In [ ]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

#the queries dataframe should have qid and query columns
tfidf_res = tfidf_retr.transform(topics)
tfidf_res.head(11)

,qid,docid,docno,rank,score,query
0,1,8172,8172,0,13.773330,measurement of dielectric constant of liquids by the use of microwave techniques
1,1,9881,9881,1,12.369113,measurement of dielectric constant of liquids by the use of microwave techniques
2,1,5502,5502,2,12.204707,measurement of dielectric constant of liquids by the use of microwave techniques
3,1,1502,1502,3,11.004179,measurement of dielectric constant of liquids by the use of microwave techniques
4,1,4871,4871,4,10.101840,measurement of dielectric constant of liquids by the use of microwave techniques
5,1,4817,4817,5,10.006881,measurement of dielectric constant of liquids by the use of microwave techniques
6,1,9859,9859,6,9.885614,measurement of dielectric constant of liquids by the use of microwave techniques
7,1,8276,8276,7,9.626918,measurement of dielectric constant of liquids by the use of microwave techniques
8,1,7234,7234,8,9.375348,measurement of dielectric constant of liquids by the use of microwave techniques
9,1,720,720,9,9.044948,measurement of dielectric constant of liquids by the use of microwave techniques


In [ ]:
qrels = vaswani_dataset.get_qrels()
print(qrels)
print(topics)

     qid  docno  label
0      1   1239      1
1      1   1502      1
2      1   4462      1
3      1   4569      1
4      1   5472      1
...   ..    ...    ...
2078  93   9875      1
2079  93   9956      1
2080  93  10497      1
2081  93  11191      1
2082  93  11318      1

[2083 rows x 3 columns]
   qid  \
0    1   
1    2   
2    3   
3    4   
4    5   
..  ..   
88  89   
89  90   
90  91   
91  92   
92  93   

                                                                                                             query  
0                                 measurement of dielectric constant of liquids by the use of microwave techniques  
1                                   mathematical analysis and design details of waveguide fed microwave radiations  
2   use of digital computers in the design of band pass filters having given phase and attenuation characteristics  
3                                                                  systems of data coding for information tran

In [ ]:
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)
res

,qid,docid,docno,rank,score,query
0,1,8172,8172,0,13.773330,measurement of dielectric constant of liquids by the use of microwave techniques
1,1,9881,9881,1,12.369113,measurement of dielectric constant of liquids by the use of microwave techniques
2,1,5502,5502,2,12.204707,measurement of dielectric constant of liquids by the use of microwave techniques
3,1,1502,1502,3,11.004179,measurement of dielectric constant of liquids by the use of microwave techniques
4,1,4871,4871,4,10.101840,measurement of dielectric constant of liquids by the use of microwave techniques
...,...,...,...,...,...,...
91833,93,8959,8959,995,4.589304,high frequency oscillators using transistors theoretical treatment and practical circuit details
91834,93,7489,7489,996,4.582137,high frequency oscillators using transistors theoretical treatment and practical circuit details
91835,93,752,752,997,4.579767,high frequency oscillators using transistors theoretical treatment and practical circuit details
91836,93,7843,7843,998,4.577280,high frequency oscillators using transistors theoretical treatment and practical circuit details


In [ ]:
eval = pt.Evaluate(res,qrels)
eval

{'map': 0.27483081354879757, 'ndcg': 0.5964474121814315}

In [ ]:
eval = pt.Evaluate(res,qrels,metrics=["map","recall","P"], perquery=True)
eval

defaultdict(dict,
            {'1': {'map': 0.25168554519046976,
              'P@5': 0.6,
              'P@10': 0.4,
              'P@15': 0.3333333333333333,
              'P@20': 0.3,
              'P@30': 0.2,
              'P@100': 0.09,
              'P@200': 0.07,
              'P@500': 0.034,
              'P@1000': 0.018,
              'R@5': 0.15789473684210525,
              'R@10': 0.21052631578947367,
              'R@15': 0.2631578947368421,
              'R@20': 0.3157894736842105,
              'R@30': 0.3157894736842105,
              'R@100': 0.47368421052631576,
              'R@200': 0.7368421052631579,
              'R@500': 0.8947368421052632,
              'R@1000': 0.9473684210526315},
             '2': {'map': 0.05706592384930798,
              'P@5': 0.2,
              'P@10': 0.1,
              'P@15': 0.06666666666666667,
              'P@20': 0.05,
              'P@30': 0.06666666666666667,
              'P@100': 0.05,
              'P@200': 0.045,
        

**GUI**

In [ ]:
%%writefile main.py
import streamlit as st
from google.colab import drive
import pandas as pd
import numpy as np
import nltk
import pyterrier as pt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.stem import *
from nltk.stem.porter import *
from collections import Counter
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
pd.set_option('display.max_colwidth', 150)
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
vaswani_dataset = pt.datasets.get_dataset("vaswani")
topics = vaswani_dataset.get_topics()
qrels = vaswani_dataset.get_qrels()
data_path = vaswani_dataset.get_corpus()[0]
with open(data_path, 'r') as file:
    documents = file.read()

documents = documents.split('<DOCNO>')
df = pd.DataFrame(documents)
# the documents stored in the vaswani_dataset
df['documents'] = df
df.drop(0, axis = 1, inplace = True)
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def preprocessing(text):
  #cleaning
    text = str(text)  # Convert text to a string
    text = re.sub(r"http\S+", " ", text)  # Remove URLs
    text = re.sub(r"RT ", " ", text)  # Remove RT
    text = re.sub(r"@[\w]*", " ", text)  # Remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text)  # Remove special characters
    text = re.sub(r'\t', ' ', text)  # Remove tabs
    text = re.sub(r'\n', ' ', text)  # Remove line jumps
    text = re.sub(r"\s+", " ", text)  # Remove extra white spaces
    text = text.strip()
    text = re.sub(r" < doc > < doc >", " ", text)
    text = re.sub(r"< docno > ", " ", text)
    tokens=word_tokenize(text)
    remove_stopwords=[word.lower()for word in tokens if word.lower()not in stop_words]
    stemmed=[stemmer.stem(word) for word in remove_stopwords]
    return " ".join(stemmed)


df['processed_documents'] = df['documents'].apply(preprocessing)
df['processed_documents']  = df['processed_documents'].replace(to_replace=r'\d', value='', regex=True)  # this line removes any digit found in the text
df['documents']  = df['documents'].replace(to_replace=r'\d', value='', regex=True)
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
def doc(df):
  df['docno']=range(len(df))
  df['docno']=df['docno'].astype(str)
  return df
df = doc(df)
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
# index the text, record the docnos as metadata
index_ref = indexer.index(df["processed_documents"], df["docno"])
index = pt.IndexFactory.of(index_ref)
index_ref.toString()
def search(q):
    query= preprocessing(q)
    bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
    rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
    rm3_qe = bm25 >> rm3_expander
    expanded_query = rm3_qe.search(query).iloc[0]["query"]
    expanded_query
    # After that you can search using the expanded query
    expanded_query_formatted = ' '.join(expanded_query.split()[1:])

    results_wqe = bm25.search(expanded_query_formatted)

    results = list(df[['processed_documents']][df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())].values)

    return results
def main():
    st.title("Discover")
    st.text("Enter your query to Discover the world")

    search_query = st.text_input("Search query:")
    submit = st.button("Search")

    if submit and search_query:
      search_results = search(search_query)

      st.write("Search results")
      for result in search_results:
        st.write(result)




if __name__ == "__main__":
    main()

Writing main.py


In [ ]:
!streamlit run main.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.43.108:8501

npx: installed 22 in 8.073s
your url is: https://big-olives-pump.loca.lt
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
11:32:50.220 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (0) - further warnings are suppressed
11:33:02.694 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_dat